In [ ]:
import os
gpu_num = "" # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["DRJIT_LIBLLVM_PATH"]="/opt/homebrew/opt/llvm@16/lib/libLLVM.dylib"

In [ ]:
resolution = [480,320] # increase for higher quality of renderings

import sionna
import tensorflow as tf

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray

# random seed fixed for repeatability
tf.random.set_seed(1)

In [ ]:
!pwd

In [ ]:
# Load integrated scene
scene = load_scene("/Users/andreamaestri/Desktop/tectwin/GEO/geoloc-rf/scenes/dusseldorf_tile_densification/dusseldorf_tile.xml") 

In [ ]:
# define TX and RX antennas and RF information

# Set the scattering coefficient of the radio material in the scene
# to a non-zero value to enable scattering
for mat in scene.radio_materials.values():
    mat.scattering_coefficient = 1./tf.sqrt(3.)

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1,
                             num_cols=16,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="V",
                             polarization_model=2)

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="dipole",
                             polarization="V",
                             polarization_model=2)

scene.frequency = 3.5e9 # set the carrier frequency, 3.5 GHz is the default
cm_cell_size = np.array([1,1]) # Each cell is 0.5mx0.5m 

In [ ]:
#place TX on top of building (with 8 meters extra)
try:
    scene.remove("tx")
except:
    print("no tx")
tx = Transmitter("tx", position=[5, -67, 16],
                orientation=[0.0,0.0,0.0])

tx.look_at([ 30, -27,  0])

scene.add(tx)

In [ ]:
scene.preview()

In [ ]:
for i in range(10):
    print(i)
    cm = scene.coverage_map(num_samples=10e5, # Reduce if your GPU does not have enough memory
                        cm_cell_size=cm_cell_size,
                        diffraction=True, scattering=False,  # Enables diffraction and scattering in addition to reflection and LoS
                        check_scene=False,
                        ) # Don't check the scene prior to compute to speed things up)

In [ ]:
scene.preview(coverage_map=cm)

In [ ]:
point_measuraments = []

In [ ]:
counter = 0
for i in cm.as_tensor()[0].numpy():
    for j in i:
        if j == 0:
            counter += 1

counter

In [ ]:
import seaborn as sns
map = 10.*np.log10(cm.as_tensor()[0].numpy().copy())
sns.heatmap(map,vmin=-200,vmax=10)
plt.show()

In [ ]:
#place TX on top of building (with 8 meters extra)
try:
    scene.remove("tx")
except:
    print("no tx")
tx = Transmitter("tx1", position=[5, -30, 16],
                orientation=[0.0,0.0,0.0])

tx.look_at([ 30, 27,  0])


scene.add(tx)

In [ ]:
cm = scene.coverage_map(num_samples=10e6, # Reduce if your GPU does not have enough memory
                        cm_cell_size=cm_cell_size,
                        diffraction=True, scattering=True,  # Enables diffraction and scattering in addition to reflection and LoS
                        check_scene=False,
                        ) # Don't check the scene prior to compute to speed things up)

In [ ]:
#map2 = 10.*np.log10(cm.as_tensor()[0].numpy())
map2 = 10.*np.log10(cm.as_tensor()[0].numpy())
print(map2[343][242])
point_measuraments.append(map2[343][242])
sns.heatmap(map2,vmin=-200,vmax=-20)
plt.show()

In [ ]:
#place TX on top of building (with 8 meters extra)
try:
    scene.remove("tx")
except:
    print("no tx")
tx = Transmitter("tx", position=[8, -70, 20],
                orientation=[0.0,0.0,0.0])

tx.look_at([ 30, -70,  0])

scene.add(tx)

In [ ]:
cm = scene.coverage_map(num_samples=10e6, # Reduce if your GPU does not have enough memory
                        cm_cell_size=cm_cell_size,
                        diffraction=True, scattering=True,  # Enables diffraction and scattering in addition to reflection and LoS
                        check_scene=False,
                        ) # Don't check the scene prior to compute to speed things up)

In [ ]:
map3 = 10.*np.log10(cm.as_tensor()[0].numpy())
print(map3[343][242])
point_measuraments.append(map3[343][242])
sns.heatmap(map3,vmin=-200,vmax=-20)
plt.show()

In [ ]:
#place TX on top of building (with 8 meters extra)
try:
    scene.remove("tx")
except:
    print("no tx")
tx = Transmitter("tx", position=[-58, 55, 16],
                orientation=[0.0,0.0,0.0])

tx.look_at([ 0, 55,  0])

scene.add(tx)
cm = scene.coverage_map(num_samples=10e6, # Reduce if your GPU does not have enough memory
                        cm_cell_size=cm_cell_size,
                        diffraction=True, scattering=True,  # Enables diffraction and scattering in addition to reflection and LoS
                        check_scene=False,
                        ) # Don't check the scene prior to compute to speed things up)
scene.preview(coverage_map=cm)
map4 = 10.*np.log10(cm.as_tensor()[0].numpy())
print(map4[343][242])
point_measuraments.append(map4[343][242])
sns.heatmap(map4,vmin=-200,vmax=-20)
plt.show()

In [ ]:
point_measuraments

In [ ]:
maps = [map, map2, map3,map4]

In [ ]:
import scipy.stats
def map_to_probability(signal, map):
    prob_map = np.zeros(np.shape(map))
    nd = scipy.stats.norm(signal,np.sqrt(np.abs(signal)))
    for idx,element in enumerate(map):
        for idx2,j in enumerate(element):
            prob_map[idx][idx2] = nd.pdf(j)

    return prob_map


In [ ]:
prob = map_to_probability(point_measuraments[0],maps[0])
prob2 = map_to_probability(point_measuraments[1],maps[1])
prob3 = map_to_probability(point_measuraments[2],maps[2])
prob5 = map_to_probability(point_measuraments[3],maps[3])

In [ ]:
print(prob5[343][242], np.max(prob5))
print(prob3[343][242], np.max(prob3))
print(prob2[343][242], np.max(prob2))
print(prob[343][242], np.max(prob))


In [ ]:
f, (ax1, ax2,ax3,ax5) = plt.subplots(1, 4,figsize=(20, 5))
f.suptitle("Probability for single antennas")
sns.heatmap(prob,vmin=0, vmax=0.07, ax=ax1, cmap="viridis")
sns.heatmap(prob2,vmin=0, vmax=0.07, ax=ax2,cmap="viridis")
sns.heatmap(prob3,vmin=0, vmax=0.07, ax=ax3, cmap="viridis")
sns.heatmap(prob5,vmin=0, vmax=0.07, ax=ax5, cmap="viridis")
plt.show()

In [ ]:
prob_final = prob2 * prob * prob3 * prob5

prob_final[343][242]

In [ ]:
sns.heatmap(prob_final)
plt.title("Location probability")
plt.axvline(242,linewidth=0.7)
plt.axhline(343,linewidth=0.7, label="real location")
plt.legend()
plt.show()